In [1]:
!pwd
import sys
%reload_ext autoreload
%autoreload 2
%matplotlib inline
sys.executable

/c/Users/81908/jupyter_notebook/tf_2_work/PyImageSearch/Bounding_box_regression


'C:\\Users\\81908\\Anaconda3\\envs\\tfgpu\\python.exe'

# Object detection: Bounding box regression with Keras, TensorFlow, and Deep Learning
- https://www.pyimagesearch.com/2020/10/05/object-detection-bounding-box-regression-with-keras-tensorflow-and-deep-learning/
- 2020/10の時点では、データとソースコード有料になってたので実行できない。ソースの流れのみ記述する

<br>

### predictのみはcolabで公開されてた
- https://colab.research.google.com/drive/1BaKbEokJF1wJ9peHnYath9Bzx2-dGbfE#scrollTo=EC3NrjWVyXZE

## config.py

In [2]:
# import the necessary packages
import os

# 入力データセットの基本パスを定義し，それを利用して画像ディレクトリと注釈CSVファイルへのパスを導出
BASE_PATH = "dataset"
IMAGES_PATH = os.path.sep.join([BASE_PATH, "images"])
ANNOTS_PATH = os.path.sep.join([BASE_PATH, "airplanes.csv"])

In [3]:
# define the path to the base output directory
BASE_OUTPUT = "output"

# 出力される直列化モデル、モデルトレーニングプロット、テスト画像ファイル名へのパスを定義
MODEL_PATH = os.path.sep.join([BASE_OUTPUT, "detector.h5"])
PLOT_PATH = os.path.sep.join([BASE_OUTPUT, "plot.png"])
TEST_FILENAMES = os.path.sep.join([BASE_OUTPUT, "test_images.txt"])

In [4]:
# initialize our initial learning rate, number of epochs to train
# for, and the batch size
INIT_LR = 1e-4
NUM_EPOCHS = 25
BATCH_SIZE = 32

## train.py
### 処理の流れは以下
- 飛行機のトレーニングデータをディスクからロードする（つまり、クラスラベルとバウンディングボックスの座標の両方）
- ディスクからVGG16をロードし（ImageNetで事前トレーニング済み）、ネットワークから完全に接続された分類レイヤーヘッドを削除し、バウンディングボックス回帰レイヤーヘッドを挿入します
- トレーニングデータのバウンディングボックス回帰レイヤーヘッドを微調整する

<br>

- $ python train.py

In [5]:
# import the necessary packages

#from pyimagesearch import config
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

In [7]:
# load the contents of the CSV annotations file
print("[INFO] loading dataset...")
#rows = open(config.ANNOTS_PATH).read().strip().split("\n")

# データ（画像）のリストを初期化し，出力予測値（バウンディングボックスの座標）を，個々の画像のファイル名とともに出力
data = []
targets = []
filenames = []

[INFO] loading dataset...


### bounding box annotations CSV data
- image_0001.jpg,49,30,349,137
- image_0002.jpg,59,35,342,153

<br>

#### アノテーションcsvの列は以下の内容記述（物体検知のみなのでラベルなし）
- Filename
- Starting x-coordinate
- Starting y-coordinate
- Ending x-coordinate
- Ending y-coordinate


In [ ]:
# loop over the rows
for row in rows:
    # break the row into the filename and bounding box coordinates
    row = row.split(",")
    (filename, startX, startY, endX, endY) = row
    
    # 入力画像へのパスを導出し，（OpenCV形式で）画像を読み込み，その寸法を取得
    imagePath = os.path.sep.join([config.IMAGES_PATH, filename])
    image = cv2.imread(imagePath)
    (h, w) = image.shape[:2]
    
    # 入力画像の空間寸法に対するバウンディングボックスの座標を拡大縮小
    startX = float(startX) / w
    startY = float(startY) / h
    endX = float(endX) / w
    endY = float(endY) / h
    
    # load the image and preprocess it
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    
    # update our list of data, targets, and filenames
    data.append(image)
    targets.append((startX, startY, endX, endY))
    filenames.append(filename)
    
# データとターゲットを NumPy 配列に変換し，入力ピクセル強度を [0, 255] から [0, 1] の範囲でスケーリング
data = np.array(data, dtype="float32") / 255.0
targets = np.array(targets, dtype="float32")

# 90%のデータを学習用に、残りの10%のデータをテスト用に使用して、データを学習用とテスト用に分割
split = train_test_split(data, targets, filenames, test_size=0.10, random_state=42)

# unpack the data split
(trainImages, testImages) = split[:2]
(trainTargets, testTargets) = split[2:4]
(trainFilenames, testFilenames) = split[4:]

# バウンディングボックスリグレグレッサーを評価/テストするときに使えるように、テスト用のファイル名をディスクに書き出します
print("[INFO] saving testing filenames...")
f = open(config.TEST_FILENAMES, "w")
f.write("\n".join(testFilenames))
f.close()

### vgg16の出力層けして、4nodeの出力層（x,yの座標）に変更

In [8]:
# load the VGG16 network, ensuring the head FC layers are left off
vgg = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# freeze all VGG layers so they will *not* be updated during the
# training process
vgg.trainable = False

# flatten the max-pooling output of VGG
flatten = vgg.output
flatten = Flatten()(flatten)

# construct a fully-connected layer header to output the predicted
# bounding box coordinates
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(4, activation="sigmoid")(bboxHead)

# construct the model we will fine-tune for bounding box regression
model = Model(inputs=vgg.input, outputs=bboxHead)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

### 座標の回帰なのでmse でfit 実行 

In [ ]:
# initialize the optimizer, compile the model, and show the model summary
opt = Adam(lr=config.INIT_LR)
model.compile(loss="mse", optimizer=opt)
print(model.summary())

# train the network for bounding box regression
print("[INFO] training bounding box regressor...")
H = model.fit(
    trainImages, trainTargets,
    validation_data=(testImages, testTargets),
    batch_size=config.BATCH_SIZE,
    epochs=config.NUM_EPOCHS,
    verbose=1)

モデル保存と学習曲線plot

In [ ]:
# serialize the model to disk
print("[INFO] saving object detector model...")
model.save(config.MODEL_PATH, save_format="h5")

# plot the model training history
N = config.NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.title("Bounding Box Regression Loss on Training Set")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.savefig(config.PLOT_PATH)

## forecast.py
- 予測実行用py

- $ python predict.py --input dataset/images/image_0697.jpg

- $ python predict.py --input output/test_images.txt

In [9]:
# import the necessary packages
#from pyimagesearch import config
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import load_model
import numpy as np
import mimetypes
import argparse
import imutils
import cv2
import os

In [ ]:
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--input", required=True, help="path to input image/text file of image filenames")
args = vars(ap.parse_args())

In [ ]:
# 入力ファイルの種類を決定、単一の入力画像を処理していると仮定
filetype = mimetypes.guess_type(args["input"])[0]
imagePaths = [args["input"]]

# ファイルタイプがテキストファイルの場合、複数の*画像を処理する必要があります。
if "text/plain" == filetype:
    
    # テストファイルのファイル名を読み込み、画像パスのリストを初期化します。
    filenames = open(args["input"]).read().strip().split("\n")
    imagePaths = []
    
    # ファイル名をループ
    for f in filenames:
        # 画像ファイル名へのフルパスを作成し、画像パスリストを更新
        p = os.path.sep.join([config.IMAGES_PATH, f])
        imagePaths.append(p)

In [ ]:
# load our trained bounding box regressor from disk
print("[INFO] loading object detector...")
model = load_model(config.MODEL_PATH)

# バウンディングボックス回帰モデルを使用してテストする画像をループ
for imagePath in imagePaths:
    
    # 入力画像（Keras形式）をディスクから読み込み，前処理を行い，ピクセル強度を[0, 1]の範囲にスケーリング
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image) / 255.0
    image = np.expand_dims(image, axis=0)
    
    # 入力画像に対してバウンディングボックス予測
    preds = model.predict(image)[0]
    (startX, startY, endX, endY) = preds
    
    # 入力画像を読み込み（OpenCV形式），画面に収まるようにリサイズし，その寸法を取
    image = cv2.imread(imagePath)
    image = imutils.resize(image, width=600)
    (h, w) = image.shape[:2]
    
    # 画像の寸法に基づいて，予測されたバウンディングボックスの座標をスケーリング
    startX = int(startX * w)
    startY = int(startY * h)
    endX = int(endX * w)
    endY = int(endY * h)
    
    # 画像上に予測される外接枠を描画
    cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)
    
    # 出力画像を表示
    cv2.imshow("Output", image)
    cv2.waitKey(0)